In [8]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [9]:
# 1. Cargar el CSV
# Le decimos que el separador es punto y coma
ruta = "../data/graph_geom_corrected_cycles.csv"
df = pd.read_csv(ruta, sep=';')

print(f"Archivo cargado. Filas encontradas: {len(df)}")

df = df.dropna(subset=['geometry'])

# 2. Convertir la columna de texto a Geometría
# Esto toma el string "LINESTRING (130...)" y lo vuelve un objeto matemático
df['geometry'] = df['geometry'].apply(wkt.loads)

# 3. Crear el GeoDataFrame
# Aquí "le avisamos" a Python que estos datos están en Metros (UTM)
# IMPORTANTE: Usa el mismo código EPSG que te funcionó en QGIS (32718 o 32719)
crs_original = "EPSG:32719" 
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=crs_original)

# 4. Reproyectar a Latitud/Longitud (WGS84)
# Este es el paso mágico. Los mapas web OBLIGATORIAMENTE necesitan EPSG:4326
print("Traduciendo coordenadas de Metros a Latitud/Longitud...")
gdf_web = gdf.to_crs(epsg=4326)

# 5. Limpieza opcional (para que el archivo pese menos)
# Si solo necesitas el ID del borde para colorearlo, guardamos solo eso y la geometría
# Asumo que la columna 'edge' es el ID único de la arista.
gdf_final = gdf_web[['edge', 'self', 'other', 'geometry']]

# 6. Guardar como GeoJSON
nombre_salida = 'grafo_base.geojson'
gdf_final.to_file(nombre_salida, driver='GeoJSON')

print(f"¡Listo! Archivo generado: {nombre_salida}")

Archivo cargado. Filas encontradas: 4862
Traduciendo coordenadas de Metros a Latitud/Longitud...
¡Listo! Archivo generado: grafo_base.geojson
